In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
df = pd.read_csv('matches.csv')
df.head()

,edition,round,score,team1,team2,url,venue,year
0,1930-URUGUAY,GROUP_STAGE,4-1 (3-0),France,Mexico (México),1930_URUGUAY_FS.htm#1-WC-30-I,Montevideo.,1930
1,1930-URUGUAY,GROUP_STAGE,3-0 (2-0),USA,Belgium (België),1930_URUGUAY_FS.htm#13-WC-30-I,Montevideo.,1930
2,1930-URUGUAY,GROUP_STAGE,2-1 (2-0),Yugoslavia (Југославија),Brazil (Brasil),1930_URUGUAY_FS.htm#7-WC-30-I,Montevideo.,1930
3,1930-URUGUAY,GROUP_STAGE,3-1 (1-0),Romania (România),Peru (Perú),1930_URUGUAY_FS.htm#10-WC-30-I,Montevideo.,1930
4,1930-URUGUAY,GROUP_STAGE,1-0 (0-0),Argentina,France,1930_URUGUAY_FS.htm#2-WC-30-I,Montevideo.,1930


In [3]:
tab = df.copy()
del tab['round']
del tab['edition']
del tab['url']
del tab['venue']

In [4]:
tab['score1'] = tab['score'].str.split(' ').str[0].str.split('-').str[0]
tab['score2'] = tab['score'].str.split(' ').str[0].str.split('-').str[1]
del tab['score']
tab.head()

,team1,team2,year,score1,score2
0,France,Mexico (México),1930,4,1
1,USA,Belgium (België),1930,3,0
2,Yugoslavia (Југославија),Brazil (Brasil),1930,2,1
3,Romania (România),Peru (Perú),1930,3,1
4,Argentina,France,1930,1,0


In [5]:
tab['team1_aux'] = tab['team1'].str.split(' ').str[0]
tab['team2_aux'] = tab['team2'].str.split(' ').str[0]
tab.head()

,team1,team2,year,score1,score2,team1_aux,team2_aux
0,France,Mexico (México),1930,4,1,France,Mexico
1,USA,Belgium (België),1930,3,0,USA,Belgium
2,Yugoslavia (Југославија),Brazil (Brasil),1930,2,1,Yugoslavia,Brazil
3,Romania (România),Peru (Perú),1930,3,1,Romania,Peru
4,Argentina,France,1930,1,0,Argentina,France


In [6]:
tab['team1'] = tab['team1_aux']
tab['team2'] = tab['team2_aux']
del tab['team1_aux']
del tab['team2_aux']
tab.head()

,team1,team2,year,score1,score2
0,France,Mexico,1930,4,1
1,USA,Belgium,1930,3,0
2,Yugoslavia,Brazil,1930,2,1
3,Romania,Peru,1930,3,1
4,Argentina,France,1930,1,0


In [7]:
tab['team1'] == 'Brazil'

0       False
1       False
2       False
3       False
4       False
5       False
6       False
7       False
8       False
9       False
10      False
11       True
12      False
13      False
14      False
15      False
16      False
17      False
18      False
19      False
20      False
21      False
22      False
23      False
24      False
25      False
26      False
27      False
28      False
29      False
        ...  
7269    False
7270    False
7271    False
7272    False
7273    False
7274    False
7275    False
7276    False
7277    False
7278    False
7279    False
7280    False
7281    False
7282    False
7283     True
7284    False
7285    False
7286    False
7287    False
7288    False
7289    False
7290    False
7291     True
7292    False
7293    False
7294    False
7295     True
7296    False
7297     True
7298    False
Name: team1, Length: 7299, dtype: bool

In [8]:
brazil_games = tab[(tab['team1'] == 'Brazil') | (tab['team2'] == 'Brazil')]  # ou tab.where(<condicional>)
brazil_games = brazil_games.dropna()
brazil_games.head()

,team1,team2,year,score1,score2
2,Yugoslavia,Brazil,1930,2,1
11,Brazil,Bolivia,1930,4,0
50,Spain,Brazil,1934,3,1
88,Brazil,Poland,1938,6,5
93,Brazil,Czechoslovakia,1938,1,1


In [16]:
bg = brazil_games
bg = bg.set_index(['year'])
bg.head()

,team1,team2,score1,score2
year,,,,
1930,Yugoslavia,Brazil,2,1
1930,Brazil,Bolivia,4,0
1934,Spain,Brazil,3,1
1938,Brazil,Poland,6,5
1938,Brazil,Czechoslovakia,1,1


In [31]:
bg['team1_aux'] = bg['team1']
bg['score1_aux'] = bg['score1']
bg.head()

,team1,team2,score1,score2,team1_aux,score1_aux
year,,,,,,
1930,Brazil,Brazil,2,1,Yugoslavia,2
1930,Brazil,Bolivia,4,0,Brazil,4
1934,Brazil,Brazil,3,1,Spain,3
1938,Brazil,Poland,6,5,Brazil,6
1938,Brazil,Czechoslovakia,1,1,Brazil,1


In [32]:
bg['team1'][bg['team1_aux'] != 'Brazil'] = bg['team2']
bg['team2'][bg['team1_aux'] != 'Brazil'] = bg['team1_aux']
bg['score1'][bg['team1_aux'] != 'Brazil'] = bg['score2']
bg['score2'][bg['team1_aux'] != 'Brazil'] = bg['score1_aux']
bg.head()

,team1,team2,score1,score2,team1_aux,score1_aux
year,,,,,,
1930,Brazil,Brazil,1,2,Yugoslavia,2
1930,Brazil,Bolivia,4,0,Brazil,4
1934,Brazil,Brazil,1,3,Spain,3
1938,Brazil,Poland,6,5,Brazil,6
1938,Brazil,Czechoslovakia,1,1,Brazil,1


In [39]:
del bg['team1_aux']
del bg['score1_aux']
bg.head()

,team1,team2,score1,score2
year,,,,
1930,Brazil,Yugoslavia,1,2
1930,Brazil,Bolivia,4,0
1934,Brazil,Spain,1,3
1938,Brazil,Poland,6,5
1938,Brazil,Czechoslovakia,1,1


In [87]:
bg['score1'] = pd.to_numeric(bg['score1'])
bg['score2'] = pd.to_numeric(bg['score2'])
bg.head()

,team1,team2,score1,score2
year,,,,
1930,Brazil,Yugoslavia,1,2
1930,Brazil,Bolivia,4,0
1934,Brazil,Spain,1,3
1938,Brazil,Poland,6,5
1938,Brazil,Czechoslovakia,1,1


In [92]:
# Média de gols de todos os adversários do Brasil de 1930 até 2014
means = bg.copy()
means.groupby(['team1', 'team2'])['score1','score2'].mean()

score1    score2
team1  team2                             
Brazil Algeria         1.000000  0.000000
       Argentina       1.600000  1.100000
       Australia       2.000000  0.000000
       Austria         2.000000  0.000000
       Belgium         2.000000  0.000000
       Bolivia         2.642857  0.857143
       Bulgaria        2.000000  0.000000
       Cameroon        3.500000  0.500000
       Chile           2.357143  0.785714
       China           4.000000  0.000000
       Colombia        1.727273  0.363636
       Costa           3.000000  1.000000
       Croatia         2.000000  0.500000
       Czechoslovakia  2.000000  0.800000
       Denmark         3.000000  2.000000
       Ecuador         1.500000  0.625000
       England         1.500000  0.500000
       France          1.500000  1.750000
       GDR             1.000000  0.000000
       Germany         1.500000  3.500000
       Ghana           3.000000  0.000000
       Hungary         1.500000  3.500000
       Italy           1.800000  1.400000
       Ivory           3.000000  1.000000
       Japan           4.000000  1.000000
       Mexico          2.750000  0.000000
       Morocco         3.000000  0.000000
       Netherlands     1.000000  2.000000
       New             4.000000  0.000000
       North           2.000000  1.000000
       Northern        3.000000  0.000000
       Norway          1.000000  2.000000
       Paraguay        1.642857  0.642857
       Peru            1.636364  0.545455
       Poland          3.250000  1.750000
       Portugal        0.500000  1.500000
       Romania         3.000000  2.000000
       Russia          2.000000  0.000000
       Scotland        1.750000  0.500000
       Soviet          2.000000  0.500000
       Spain           2.000000  1.000000
       Sweden          3.000000  1.142857
       Switzerland     2.000000  2.000000
       Turkey          1.500000  0.500000
       USA             1.000000  0.000000
       Uruguay         1.800000  1.100000
       Venezuela       4.071429  0.214286
       Wales           1.000000  0.000000
       Yugoslavia      1.000000  0.750000
       Zaire           3.000000  0.000000